# Funções úteis


In [1]:
# @title Derivadas Numéricas
import numpy as np


def dydz(y: np.ndarray, dz: float) -> np.ndarray:
    """
    Computes the first derivative of y with respect to z.
    """
    dydz = np.zeros_like(y)

    dydz[0] = (-3 * y[0] + 4 * y[1] - y[2]) / (2 * dz)  # Forward at inlet
    dydz[1:-1] = (y[2:] - y[:-2]) / (2 * dz)  # Central
    dydz[-1] = (3 * y[-1] - 4 * y[-2] + y[-3]) / (2 * dz)  # Backward at outlet
    return dydz


def d2ydz2(y: np.ndarray, dz: float) -> np.ndarray:
    d2ydz2 = np.zeros_like(y)

    d2ydz2[0] = (2 * y[0] - 5 * y[1] + 4 * y[2] - y[3]) / dz**2  # Forward at inlet
    d2ydz2[1:-1] = (y[2:] - 2 * y[1:-1] + y[:-2]) / dz**2  # Central
    d2ydz2[-1] = (
        2 * y[-1] - 5 * y[-2] + 4 * y[-3] - y[-4]
    ) / dz**2  # Backward at outlet
    return d2ydz2


In [2]:
# @title Empacotar e desempacotar y


def pack(y_dict):
    parts = []
    parts.append(y_dict["Cg"].reshape(-1))  # (ng, n_z)
    parts.append(y_dict["q"].reshape(-1))  # (ng, n_z)
    parts.append(y_dict["Tg"].reshape(-1))  # (n_z,)
    parts.append(y_dict["Tp"].reshape(-1))  # (n_z,)
    parts.append(y_dict["Tw"].reshape(-1))  # (n_z,)
    parts.append(y_dict["Csi"].reshape(-1))  # (ng, n_z)
    parts.append(y_dict["Pmpbar"].reshape(-1))  # (ng, n_z)
    parts.append(y_dict["qasti"].reshape(-1))  # (ng, n_z)
    parts.append(y_dict["yi"].reshape(-1))  # (ng, n_z)
    parts.append(y_dict["Cgt"].reshape(-1))  # (n_z,)
    parts.append(y_dict["P"].reshape(-1))  # (n_z,)
    parts.append(y_dict["u0"].reshape(-1))  # (n_z,)
    return np.concatenate(parts)


def unpack(y, ng, n_z):
    nt = y.shape[0]
    offset = 0
    out = {}

    out["Cg"] = y[:, offset : offset + ng * n_z].reshape(nt, ng, n_z)
    offset += ng * n_z

    out["q"] = y[:, offset : offset + ng * n_z].reshape(nt, ng, n_z)
    offset += ng * n_z

    out["Tg"] = y[:, offset : offset + n_z].reshape(nt, n_z)
    offset += n_z

    out["Tp"] = y[:, offset : offset + n_z].reshape(nt, n_z)
    offset += n_z

    out["Tw"] = y[:, offset : offset + n_z].reshape(nt, n_z)
    offset += n_z

    out["Csi"] = y[:, offset : offset + ng * n_z].reshape(nt, ng, n_z)
    offset += ng * n_z

    out["Pmpbar"] = y[:, offset : offset + ng * n_z].reshape(nt, ng, n_z)
    offset += ng * n_z

    out["qasti"] = y[:, offset : offset + ng * n_z].reshape(nt, ng, n_z)
    offset += ng * n_z

    out["yi"] = y[:, offset : offset + ng * n_z].reshape(nt, ng, n_z)
    offset += ng * n_z

    out["Cgt"] = y[:, offset : offset + n_z].reshape(nt, n_z)
    offset += n_z

    out["P"] = y[:, offset : offset + n_z].reshape(nt, n_z)
    offset += n_z

    out["u0"] = y[:, offset : offset + n_z].reshape(nt, n_z)
    offset += n_z

    return out


In [3]:
# @title Plot
import matplotlib.pyplot as plt


def plot(t, var, z, ylabel=""):
    n_z = z.shape[0]
    indices = sorted({0, n_z - 1} | set(range(0, n_z, 10)))

    plt.figure()

    if var.ndim == 2:
        # caso 1D (n_z)
        for i in indices:
            plt.plot(t, var[:, i], label=f"Z = {z[i]:.2f}")

    plt.xlabel("t")
    plt.ylabel(ylabel)
    plt.legend()
    plt.show()


# Simulate PSA


In [4]:
# @title Constantes
from typing import Final, Union

NUMBER = Union[float, np.ndarray]

ng: Final[int] = 3

Rg: Final[float] = 8.314
Rp: Final[float] = 1e-3
roap: Final[float] = 549
Cps: Final[float] = 1200
epsb: Final[float] = 0.352
rob: Final[float] = (1 - epsb) * roap
apm: Final[float] = 3 / Rp
dp: Final[float] = 2 * Rp
L: Final[float] = 0.323
Dw: Final[float] = 0.0211
row: Final[float] = 8238
Cpw: Final[float] = 500
alfaw: Final[float] = 457
alfawl: Final[float] = 499
Qfeed_SLPM: Final[float] = 0.43
bed_area: Final[float] = 3.1415 * (Dw / 2) ** 2
nocycles: Final[float] = 20
betapeq: Final[float] = 0.04
betadepco: Final[float] = 0.07
betablow: Final[float] = 0.018
dax: Final[float] = 6.9e-5
kf: Final[float] = 2.0e-2
λ: Final[float] = 0.6
hf: Final[float] = 128
visc: Final[float] = 1.7e-5
U: Final[float] = 37
hw: Final[float] = 50
Cpmix: Final[float] = 31
Cvmix: Final[float] = 23
Mwi: Final[np.ndarray] = np.array([4.400e-02, 2.8e-02, 2e-3])
dpi: Final[np.ndarray] = np.array([7.7e-7, 9.8e-7, 2.5e-6])
MDH1: Final[np.ndarray] = np.array([21885, 13351, 35260])
kinf: Final[np.ndarray] = np.array([5.78e-5, 3.36e-4, 9.39e-7])
qsat: Final[np.ndarray] = np.array([8.509, 3.482, 0.294])
Cvi: Final[np.ndarray] = np.array([29.8, 20.6, 12.5])
eta: Final[float] = 0.85
# gama: Final[float] = cpmix / cvmix

yin_feed: Final[np.ndarray] = np.array([0.3, 0.22, 0.48])
yin_peq: Final[np.ndarray] = np.array([0, 1, 0])
yin_press: Final[np.ndarray] = np.array([0, 0, 1])
yin_purge: Final[np.ndarray] = np.array([0, 0.32, 0.68])
yin_rinse: Final[np.ndarray] = np.array([1, 0, 0])
yin_rr: Final[np.ndarray] = np.array([1, 0, 0])
Phigh: Final[float] = 4.8e5
Plow: Final[float] = 0.55e5
Pblow: Final[float] = 1e5
Prinse: Final[float] = 1e5
Ppeqd: Final[float] = 33e5
Ppeqp: Final[float] = 11e5
Ppurge: Final[float] = 1e5
Ppress: Final[float] = 4e5
Pdepco: Final[float] = 1e5
Performance: Final[float] = 100
Pinter1: Final[float] = 1.5e5
Pinter2: Final[float] = 11e5
qrr: Final[float] = 1e-5
qeq: Final[float] = 0.3
countcycle: Final[float] = 1
Qn_r: Final[float] = 0.57
Qn_p: Final[float] = 0.305
Tfeed: Final[float] = 679.462
Tpress: Final[float] = 0
Teq: Final[float] = 0
Tpurge: Final[float] = 80
Trinse: Final[float] = 187
Tdepco: Final[float] = 0
Tinlet: Final[float] = 305.059

u0inlet = (Qfeed_SLPM * 100000 / Phigh * Tinlet / 273.15 / 60 / 1000) / bed_area
Qfeed = u0inlet * bed_area
Cinlet = Phigh / Rg / Tinlet
Tinf = Tinlet


In [5]:
# @title Configurações

n_z = 41  # Number of discrete points in space
z = np.linspace(0, L, n_z)  # Spatial grid
dz = z[1] - z[0]  # Spatial step size

n_t = 1000  # Number of discrete points in time
tf = 679.462  # Final simulation time
t = np.linspace(0, tf, n_t)
dt = t[1] - t[0]


In [6]:
# @title Modelo
def partial(y):
    return dydz(y, dz)


def partial2(y):
    return d2ydz2(y, dz)


def model(t, y, yp0):
    print("Solving for t=", t)

    # --- Variáveis diferenciais ---
    offset = 0
    # Concentração no gás
    Cgi = y[offset : offset + ng * n_z].reshape(ng, n_z)
    yp_Cgi = yp0[offset : offset + ng * n_z].reshape(ng, n_z)
    offset += ng * n_z

    # Concentração adsorvida
    q = y[offset : offset + ng * n_z].reshape(ng, n_z)
    yp_q = yp0[offset : offset + ng * n_z].reshape(ng, n_z)
    offset += ng * n_z

    # Temperatura do gás
    Tg = y[offset : offset + n_z]
    yp_Tg = yp0[offset : offset + n_z]
    offset += n_z

    # Temperatura das partículas
    Tp = y[offset : offset + n_z]
    yp_Tp = yp0[offset : offset + n_z]
    offset += n_z

    # Temperatura da parede
    Tw = y[offset : offset + n_z]
    yp_Tw = yp0[offset : offset + n_z]
    offset += n_z

    # --- Variáveis Algébricas ---
    Csi = y[offset : offset + ng * n_z].reshape(ng, n_z)
    offset += ng * n_z

    Pmpbar = y[offset : offset + ng * n_z].reshape(ng, n_z)
    offset += ng * n_z

    qasti = y[offset : offset + ng * n_z].reshape(ng, n_z)
    offset += ng * n_z

    P = y[offset : offset + n_z]
    offset += n_z

    u0 = y[offset : offset + n_z]
    offset += n_z

    # --- Equações do Sistema ---
    # Se no res aparecer um 999, significa que alguma equação não foi definida.
    # Ou seja, existem mais variaveis que equações. Se tudo estiver correto, todo os 999 serão substituidos
    res = np.ones_like(y) * 999
    offset = 0

    Cgt = Cgi.sum(axis=0)

    yi = np.zeros_like(Cgi)
    for i in range(ng):
        yi[i] = Cgi[i] / Cgt
        offset += n_z - 2

    Mw = np.dot(np.array(Mwi), yi)

    # Equação de Darcy (eu acho)
    res[offset : offset + n_z] = -partial(P) - (
        150 * visc * (1 - epsb) ** 2 / (epsb**3 * dp**2) * u0
        + 1.75 * (1 - epsb) / (epsb**3 * dp) * Cgt * Mw * u0 * np.abs(u0)
    )
    offset += n_z

    res[offset : offset + n_z - 2] = (Cgt * Tg * Rg - P)[1:-1]
    offset += n_z - 2

    keq = np.zeros_like(Csi)
    for i in range(ng):
        keq[i, :] = kinf[i] * np.exp(MDH1[i] / (Rg * Tp))

    # Balanço de massa
    for i in range(ng):
        # O -2 é pq a equação não é valida nas bordas, segundo o código do gPROMS
        res[offset : offset + n_z - 2] = (
            (epsb * dax * (partial(Cgt) * partial(yi[i]) + Cgt * partial2(yi[i])))
            - partial(u0 * Cgi[i])
            - epsb * yp_Cgi[i]
            - (1 - epsb) * apm * kf * (Cgi[i] - Csi[i])
        )[1:-1]
        offset += n_z - 2

    res[offset : offset + n_z - 2] = (
        (λ * partial2(Tg))
        - u0 * Cgt * Cpmix * partial(Tg)
        + epsb * Rg * Tg * yp_Cgi.sum(axis=0)
        - (1 - epsb) * apm * hf * (Tg - Tp)
        - 4 * hw / Dw * (Tg - Tw)
        - epsb * Cgt * Cvmix * yp_Tg
    )[1:-1]
    offset += n_z - 2

    for i in range(ng):
        res[offset : offset + n_z] = (
            15 * dpi[i] / P * Phigh / Rp**2 * (qasti[i] - q[i]) - yp_q[i]
        )
        offset += n_z

    for i in range(ng):
        res[offset : offset + n_z] = Csi[i] * Rg * Tp / 1e5 - Pmpbar[i]
        offset += n_z

        res[offset : offset + n_z] = 15 * dpi[i] / P * Phigh / (Rp**2) * (
            qasti[i] - q[i]
        ) - (apm * kf * (Cgi[i] - Csi[i]) / roap)
        offset += n_z

    for i in range(ng):
        res[offset : offset + n_z] = (
            qsat[i]
            * (keq[i] * Pmpbar[i])
            / (1 + (np.sum(np.abs(keq * Pmpbar), axis=0)))
            - qasti[i]
        )
        offset += n_z

    res[offset : offset + n_z] = (
        rob * (np.sum(MDH1[:, None] * yp_q))
        + (1 - epsb) * apm * hf * (Tg - Tp)
        - (1 - epsb) * (roap * np.sum(q * Cvi[:, None], axis=0) + roap * Cps) * yp_Tp
    )
    offset += n_z

    res[offset : offset + n_z] = (
        alfaw * hw * (Tg - Tw) - alfawl * U * (Tw - Tinf) - row * Cpw * yp_Tw
    )
    offset += n_z

    # --- Condições de Contorno (Feed) ---
    res[offset] = Cgi[:, 0].sum() - Cgt[0]
    offset += 1
    res[offset] = Cgi[:, -1].sum() - Cgt[-1]
    offset += 1

    for i in range(ng):
        # Inlet
        res[offset] = yin_feed[i] * Cinlet - (
            Cgi[i, 0] - epsb * dax / u0inlet * partial(Cgi[i])[0]
        )
        offset += 1

        res[offset] = Cgi[i, 0] - yi[i, 0] * Cgt[0]
        offset += 1

        # Outlet
        res[offset] = partial(Cgi[i])[-1]
        offset += 1

        res[offset] = Cgi[i, -1] - yi[i, -1] * Cgt[-1]
        offset += 1

    if P[-1] < Phigh:
        res[offset] = u0[0] * Cgt[0] - 0.5 * u0inlet * Cinlet
        offset += 1
        res[offset] = u0[-1]
        offset += 1
    else:
        res[offset] = P[-1] - Phigh
        offset += 1
        res[offset] = u0[0] * Cgt[0] - u0inlet * Cinlet
        offset += 1

    res[offset] = Tinlet - (Tg[0] - λ / (u0inlet * Cinlet * Cpmix) * partial(Tg)[0])
    offset += 1

    res[offset] = partial(Tg)[-1]
    offset += 1

    # print(Tg)
    # print("res shape:", np.asarray(np.shape(res)))

    return res


In [7]:
# @title Condições Iniciais
# from scipy.optimize import fsolve
from scipy.optimize import minimize

y0_dict = {}

y0_dict["Cgt"] = np.full(n_z, Phigh / (Rg * Tinlet))
y0_dict["yi"] = np.zeros((ng, n_z))
y0_dict["yi"][2, :] = 1.0
y0_dict["Tg"] = np.full(n_z, Tinlet)

# print("Tg:", y0_dict["Tg"])
# print("Tg d1:", partial(y0_dict["Tg"]))
# print("Tg d2:", partial2(y0_dict["Tg"]))

y0_dict["q"] = np.zeros((ng, n_z))
y0_dict["q"][0, 0] = 1.6321642
y0_dict["q"][1, 0] = 0.09842706
y0_dict["q"][2, 0] = 0.81742734

y0_dict["Tp"] = np.full(n_z, Tinlet)
y0_dict["Tw"] = np.full(n_z, Tinlet)

y0_dict["Cg"] = np.zeros((ng, n_z))
y0_dict["Cg"][0, 0] = 30.009283
y0_dict["Cg"][1, 0] = 22.006807
y0_dict["Cg"][2, 0] = 137.2389
y0_dict["Cg"][2, 1:] = 189.255

y0_dict["Pmpbar"] = np.zeros((ng, n_z))
y0_dict["Pmpbar"][0, :] = 0.01
y0_dict["Pmpbar"][1, :] = 0.01
y0_dict["Pmpbar"][2, :] = 4.8
y0_dict["Pmpbar"][0, 0] = 0.76111364
y0_dict["Pmpbar"][1, 0] = 0.55815
y0_dict["Pmpbar"][2, 0] = 3.4807365

y0_dict["Csi"] = (y0_dict["Pmpbar"] * 1e5) / (Rg * y0_dict["Tp"])

y0_dict["qasti"] = np.zeros((ng, n_z))
y0_dict["qasti"][0, 0] = 1.6321642
y0_dict["qasti"][1, 0] = 0.09842706
y0_dict["qasti"][2, 0] = 0.81742734
y0_dict["qasti"][2, 1:] = 1.4453467

y0_dict["P"] = np.ones(n_z) * 480000.16
y0_dict["u0"] = np.zeros(n_z) * 0.0001
y0_dict["u0"][0] = u0inlet
y0_dict["u0"][1] = 0.0020161946

y0 = pack(y0_dict)


# Achar derivadas iniciais com o fsolve
def residual(yp):
    return model(t[0], y0, yp)[0]


yp0_guess = np.ones_like(y0)

res = minimize(residual, np.zeros_like(y0))
print("Convergiu:", res.success, "| ||res|| =", np.linalg.norm(res.fun))
yp0 = res.x

res = model(t[0], y0, yp0)

# print("y0 shape:", np.shape(y0))
# print("yp0 shape:", np.shape(yp0))
# print("res shape:", np.shape(res))
# print("t shape:", np.shape(t))

np.set_printoptions(threshold=np.inf)  # type: ignore

# print(yp0)
print(res)


Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for 

/tmp/ipykernel_418785/1599706789.py:108: RuntimeWarning: divide by zero encountered in divide
  15 * dpi[i] / P * Phigh / Rp**2 * (qasti[i] - q[i]) - yp_q[i]
/tmp/ipykernel_418785/1599706789.py:108: RuntimeWarning: invalid value encountered in multiply
  15 * dpi[i] / P * Phigh / Rp**2 * (qasti[i] - q[i]) - yp_q[i]
/tmp/ipykernel_418785/1599706789.py:116: RuntimeWarning: divide by zero encountered in divide
  res[offset : offset + n_z] = 15 * dpi[i] / P * Phigh / (Rp**2) * (
/tmp/ipykernel_418785/1599706789.py:116: RuntimeWarning: invalid value encountered in multiply
  res[offset : offset + n_z] = 15 * dpi[i] / P * Phigh / (Rp**2) * (


Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for 

In [8]:
# @title Simulação
from scipy_dae.integrate import solve_dae

y = solve_dae(model, (t[0], t[-1]), y0, yp0, t_eval=t).y

sol = unpack(y, ng, n_z)

plot(t, sol["Tg"], z, ylabel="Gas Temperature")


Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for 

/tmp/ipykernel_418785/1599706789.py:108: RuntimeWarning: divide by zero encountered in divide
  15 * dpi[i] / P * Phigh / Rp**2 * (qasti[i] - q[i]) - yp_q[i]
/tmp/ipykernel_418785/1599706789.py:108: RuntimeWarning: invalid value encountered in multiply
  15 * dpi[i] / P * Phigh / Rp**2 * (qasti[i] - q[i]) - yp_q[i]
/tmp/ipykernel_418785/1599706789.py:116: RuntimeWarning: divide by zero encountered in divide
  res[offset : offset + n_z] = 15 * dpi[i] / P * Phigh / (Rp**2) * (
/tmp/ipykernel_418785/1599706789.py:116: RuntimeWarning: invalid value encountered in multiply
  res[offset : offset + n_z] = 15 * dpi[i] / P * Phigh / (Rp**2) * (


Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for 

/home/silas/workspace/PRH/PSA-fail/.pixi/envs/default/lib/python3.14/site-packages/scipy/integrate/_ivp/common.py:347: RuntimeWarning: invalid value encountered in subtract
  diff = f_new - f[:, None]


Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for t= 0.0
Solving for 

ValueError: array must not contain infs or NaNs